In [210]:
# TODO: Remove this before submission
%load_ext jupyternotify
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


<IPython.core.display.Javascript object>

In [154]:
import pandas as pd
import numpy as np
import nltk

nltk.download("wordnet")
import re
from bs4 import BeautifulSoup


[nltk_data] Downloading package wordnet to /Users/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [155]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

## Read Data


## Keep Reviews and Ratings


In [156]:
# Load the tab separated data file, and print the first 5 rows for confirmation
data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])
data.head()


/var/folders/d7/gcxbjppx70qfkdwy32kg04dh0000gn/T/ipykernel_6562/949400013.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


Understanding Data


In [157]:
data.describe()


,star_rating,review_body
count,1767042,1766807
unique,11,1618522
top,5,Love it
freq,1041056,4288


In [158]:
data.star_rating.unique()


array([5, 1, 4, 3, 2, nan, '5', '1', '3', '4', '2', '2012-12-21'],
      dtype=object)

In [159]:
data.groupby(["star_rating"]).count()


,review_body
star_rating,
1,150441
2,97259
3,153660
4,259019
5,1040896
1,4566
2,3541
2012-12-21,0
3,5999


In [160]:
# Drop the outlier which is star_rating = "2012-12-21"
data = data[data.star_rating != "2012-12-21"]


In [161]:
# Remove nan valued rows
data = data[data.star_rating.notnull()]


In [162]:
data.describe()


,star_rating,review_body
count,1767041,1766807
unique,10,1618522
top,5,Love it
freq,1041056,4288


In [163]:
# Convert all star rating to integer
data["star_rating"] = data.star_rating.astype(int)


In [164]:
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [165]:
data = data[data.review_body.notnull()]
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [166]:
# There are no empty reviews
(data.review_body.str.len() <= 0).sum()


0

Now we can continue with the process.


## We select 20000 reviews randomly from each rating class.


In [167]:
# np.random.seed(101)
N_SAMPLES = 25000


In [168]:
sampled_data = data.groupby("star_rating", group_keys=False).apply(lambda x: x.sample(N_SAMPLES))


# Data Cleaning


In [169]:
avg_len_before_cleaning = sampled_data.review_body.str.len().mean()
f"Avg. length of reviews BEFORE CLEANING :: {avg_len_before_cleaning}"


'Avg. length of reviews BEFORE CLEANING :: 188.868096'

In [170]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: pd.Series):
    return data.str.lower()


In [171]:
def remove_accented_characters(data: pd.Series):
    import unicodedata

    """Removes accented characters from the Series

    Args:
        data (pd.Series): Series of string

    Returns:
        _type_: pd.Series
    """
    import unicodedata

    return data.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore"))


In [172]:
def remove_html_and_url(data: pd.Series):
    """Function to remove
             1. HTML encodings
             2. HTML tags (both closed and open)
             3. URLs

    Args:
        data (pd.Series): A Pandas series of type string

    Returns:
        _type_: pd.Series
    """
    # Remove HTML encodings
    data.str.replace(r"&#\d+;", " ", regex=True)

    # Remove HTML tags (both open and closed)
    data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

    # Remove URLs
    data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)

    return data


In [173]:
! pip install emot

In [174]:
# Handle emoji
def convert_emoji_to_txt(data: pd.Series):
  from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

  EMO_TO_TXT_DICT = dict()
  for emot in UNICODE_EMOJI:
    EMO_TO_TXT_DICT[emot] = f" {re.sub(r',|:|_', '', UNICODE_EMOJI[emot])} "

  for emo in EMOTICONS_EMO:
    EMO_TO_TXT_DICT[emot] = f" {re.sub(r',| ', '', EMOTICONS_EMO[emo])} "

  def convert_emojis(text, emo_to_txt_dict):
    for emot in emo_to_txt_dict:
        text = text.replace(emot, emo_to_txt_dict[emot])
    return text

  return data.apply(lambda x: convert_emojis(x, EMO_TO_TXT_DICT))

In [175]:
# Remove non-alphabetical characters
def remove_non_alpa_characters(data: pd.Series):
    return data.str.replace(r"_+|\\|[^a-zA-Z\s]", " ", regex=True)


In [176]:
# Remove extra spaces
def remove_extra_spaces(data: pd.Series):
    return data.str.replace(r"^\s*|\s\s*", " ", regex=True)


In [177]:
# Install contractions package, if you don't have it
! pip install contractions

In [178]:
# Expanding contractions
def fix_contractions(data: pd.Series):
    import contractions

    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])

    return data.apply(contraction_fixer)


In [179]:
# A dictionary containing the columns and a list of functions to perform on it in order
data_cleaning_pipeline = {
    "review_body": [
        to_lower,
        convert_emoji_to_txt,
        remove_accented_characters,
        remove_html_and_url,
        fix_contractions,
        remove_non_alpa_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = sampled_data.copy()

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = cleaned_data[col].copy()

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    cleaned_data[col] = temp_data.copy()


Starting: to_lower
Ended: to_lower
Starting: convert_emoji_to_txt
Ended: convert_emoji_to_txt
Starting: remove_accented_characters
Ended: remove_accented_characters
Starting: remove_html_and_url
Ended: remove_html_and_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpa_characters
Ended: remove_non_alpa_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [180]:
avg_len_after_cleaning = cleaned_data.review_body.str.len().mean()
f"Avg. length of reviews after cleaning :: {avg_len_after_cleaning}"


'Avg. length of reviews after cleaning :: 184.475832'

In [181]:
f"Before Cleaning: {avg_len_before_cleaning} ;; After Cleaning: {avg_len_after_cleaning}"


'Before Cleaning: 188.868096 ;; After Cleaning: 184.475832'

In [182]:
# TODO: Remove Test Block
# Checkpoint: Cleaned Data
cleaned_data.to_csv("./data/cleaned.tsv", sep="\t", index=False, encoding="UTF-8")


# Pre-processing


## remove the stop words


In [183]:
# TODO: Remove Test Block
cleaned_data = pd.read_csv("./data/cleaned.tsv", sep="\t")


In [184]:
avg_len_before_preprocessing = cleaned_data["review_body"].str.len().mean()
print(f"Avg. length of the reviews before preprocessing :: {avg_len_before_preprocessing}")


Avg. length of the reviews before preprocessing :: 184.475832


In [185]:
def tokenize(data: pd.Series):
    from nltk.tokenize import word_tokenize

    nltk.download("punkt")

    return data.apply(word_tokenize)


In [186]:
from typing import List, Set


def remove_stopwords(data: pd.Series):
    """Remove stop words using the NLTK stopwords dictionary

    Args:
        string (str): a document

    Returns:
        str: a document with stopwords removed
    """
    from nltk.corpus import stopwords

    nltk.download("stopwords")

    stopwords = set(stopwords.words())

    def remover(word_list: List[str], stopwords: Set[str]):
        return [word for word in word_list if not word in stopwords]

    return data.apply(lambda word_list: remover(word_list, stopwords))


## perform lemmatization


In [187]:
def lemmatize(data: pd.Series, consider_pos_tag: bool = True):
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer

    nltk.download("omw-1.4")

    # POS tagging
    def perform_nltk_pos_tag(data: pd.Series):
        from nltk import pos_tag

        nltk.download("averaged_perceptron_tagger")

        return data.apply(pos_tag)

    # Convert POS tag to wordnet pos tags
    def wordnet_pos_tagger(tag: str):
        if tag.startswith("J"):
            return wordnet.ADJ
        elif tag.startswith("V"):
            return wordnet.VERB
        elif tag.startswith("N"):
            return wordnet.NOUN
        elif tag.startswith("R"):
            return wordnet.ADV
        else:
            return None

    lemmatizer = WordNetLemmatizer()
    lemmatized = list()

    if consider_pos_tag:
        pos_tagged_data = data.copy()
        pos_tagged_data = perform_nltk_pos_tag(data)

        for row in pos_tagged_data:

            lemmatized_row = list()

            if consider_pos_tag:
                for word, tag in row:
                    wordnet_pos_tag = wordnet_pos_tagger(tag)

                    if wordnet_pos_tag is None:
                        lemmatized_row.append(word)
                    else:
                        result = lemmatizer.lemmatize(word, wordnet_pos_tag)
                        lemmatized_row.append(lemmatizer.lemmatize(word, wordnet_pos_tag))

            lemmatized.append(lemmatized_row)
    else:
        for row in data:
            lemmatized_row = list()

            for word in row:
                lemmatized_row.append(lemmatizer.lemmatize(word))

            lemmatized.append(lemmatized_row)

    return pd.Series(lemmatized)


In [188]:
# Concatenate lemmatized sentences back into one sentence
def concatenate(data: pd.Series):
    return data.apply(lambda words: " ".join(words))


In [208]:
preprocessing_pipeline = {"review_body": [tokenize, lemmatize, concatenate]}

# Run the pipeline
preprocessed_data = cleaned_data.copy()

# Process all the cleaning instructions
for col, pipeline in preprocessing_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = preprocessed_data[col].copy()

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")

        if func.__name__ == "lemmatize":
            temp_data = func(temp_data, consider_pos_tag=True)
        else:
            temp_data = func(temp_data)

        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    preprocessed_data[col] = temp_data.copy()


Starting: tokenize


[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ended: tokenize
Starting: lemmatize


[nltk_data] Downloading package omw-1.4 to /Users/aditya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aditya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Ended: lemmatize
Starting: concatenate
Ended: concatenate


In [209]:
preprocessed_data.head()

,star_rating,review_body
0,1,my daughter be so excited to get this charm to...
1,1,very ugly not worth it
2,1,the first pair break while tighten the screw t...
3,1,i buy beacuse i though bracelet will so fasion...
4,1,return it


In [211]:
# TODO: Remove Test Block
# Save lemmatized data
import pickle as pkl

with open("./data/preprocessed.pkl", "wb") as file:
    pkl.dump(preprocessed_data, file)


In [212]:
avg_len_after_preprocessing = preprocessed_data["review_body"].str.len().mean()
print(f"Avg. length of the reviews after preprocessing :: {avg_len_after_preprocessing}")


Avg. length of the reviews after preprocessing :: 176.034712


In [213]:
f"Before Preprocessing: {avg_len_before_preprocessing} ;; After Preprocessing: {avg_len_after_preprocessing}"


'Before Preprocessing: 184.475832 ;; After Preprocessing: 176.034712'

# TF-IDF Feature Extraction


In [ ]:
# TODO: Remove Test Block
# Load lemmatized data
# import pickle as pkl

# preprocessed_data = None
# with open("./data/preprocessed.pkl", "rb") as file:
#     lemmatized_data = pkl.load(file)


In [214]:
preprocessed_data[preprocessed_data["review_body"].str.len() == 0].groupby("star_rating").count()


,review_body
star_rating,
1,3
2,3
3,8
4,4
5,5


In [215]:
preprocessed_data[preprocessed_data["review_body"].isnull()]


,star_rating,review_body


In [216]:
preprocessed_data["review_body"].isnull().values.any(), preprocessed_data["review_body"].isnull().sum()


(False, 0)

In [217]:
# Drop empty strings
preprocessed_data = preprocessed_data[preprocessed_data["review_body"].str.len() != 0]
# Drop NA reviews
preprocessed_data.dropna(subset=["review_body"], inplace=True)


In [218]:
preprocessed_data[preprocessed_data["review_body"].str.len() == 0].groupby("star_rating").count()


,review_body
star_rating,


In [219]:
# TODO: Remove Checkpoint ...
preprocessed_data.to_csv("./data/data.tsv", sep="\t", index=False)


In [ ]:
# TODO: Remove this block
# Retrieve checkpoint
# data = pd.read_csv("./data/data.tsv", sep="\t")


In [220]:
# Resample data.
data = preprocessed_data.groupby("star_rating", group_keys=False).apply(lambda x: x.sample(20000))


In [221]:
# Split the data 80-20 split
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, stratify=data["star_rating"])


In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(train["review_body"])

X_tfidf_train = vectorizer.transform(train["review_body"])
X_tfidf_test = vectorizer.transform(test["review_body"])
y_train = train["star_rating"]
y_test = test["star_rating"]


TfidfVectorizer()

### Helper Functions


In [224]:
def calc_metrics(y_true, y_pred):
    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

    accuracy_score = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    print(f"{accuracy_score}")

    for rating_precision, rating_recall, rating_f1 in zip(precision, recall, f1):
        print(f"{rating_precision},{rating_recall},{rating_f1}")

    print(f"{np.mean(precision)},{np.mean(recall)},{np.mean(f1)}")


# Perceptron


In [225]:
from sklearn.linear_model import Perceptron

clf = Perceptron()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


Perceptron()

0.4314
0.5001127395715896,0.5545,0.5259039715471251
0.3364856619890177,0.27575,0.30310524869469635
0.3399548532731377,0.3765,0.3572953736654805
0.3776877531941415,0.303,0.3362463587182688
0.5570137693631669,0.64725,0.5987511563367252
0.4222509554782107,0.4314,0.4242604217924592


# SVM


In [226]:
from sklearn.svm import LinearSVC

clf = LinearSVC(dual=True)
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


LinearSVC()

0.5119
0.5705547352879139,0.67625,0.6189223201006749
0.41133077604012985,0.3485,0.37731763432128845
0.42947067238912734,0.37525,0.40053368912608406
0.46151731720725675,0.41975,0.439643885833988
0.6246569558792485,0.73975,0.6773492045324483
0.4995060913607353,0.5119,0.5027533467828966


# Logistic Regression


In [227]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver="lbfgs", max_iter=500, multi_class="multinomial")

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


LogisticRegression(max_iter=500, multi_class='multinomial')

0.5359
0.6114234315268721,0.67975,0.6437788563987215
0.43661186960156034,0.39175,0.41296613519567793
0.43896713615023475,0.42075,0.42966556037784026
0.4880483320199632,0.4645,0.47598309209683615
0.6687485542447374,0.72275,0.6947014297729184
0.5287598647086736,0.5359,0.5314190147683988


# Naive Bayes


In [228]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)


MultinomialNB()

0.5069
0.6175208280737188,0.6115,0.6144956663735712
0.40718105423987777,0.39975,0.4034313107102308
0.4057290443766142,0.432,0.41845259716672717
0.4487791932059448,0.42275,0.43537590113285274
0.6545899632802937,0.6685,0.6614718614718615
0.5067600166352898,0.5069,0.5066454673710487
